In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import abatement, techdata_to_tree, sys, ShockFunction
os.chdir(directory['curr'])
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

## 1: Set up tree

*Read data:*

In [2]:
inputfile = "techdata_dors_2.xlsx" # file
inputoth  = 'Othdata_dors_2.xlsx'
output = techdata_to_tree.load_techcats(pd.read_excel(directory['data'] + "/" + inputfile, sheet_name=["inputdisp", "endofpipe", "inputprices"]))
modules = ["ID", "EOP"] # modules to use

*Set up trees:*

In [3]:
nts = {m: nesting_tree.nesting_tree(name=m) for m in modules}

#### 1.1: Input displacing tree

In [4]:
nts["ID"].add_tree(output["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":"CES_norm"}) # E to C
nts["ID"].add_tree(output["ID"]["components"], tree_name = "ID_CU", **{"type_f":"MNL"}) # C to U
nts["ID"].add_tree(output["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # U to T
nts["ID"].add_tree(output["ID"]["techs_inputs"], tree_name="ID_TX") # T to inputs X
nts["ID"].add_tree(output["ID"]["basetechs"], tree_name="ID_BU", **{"type_io":"output", "type_f":"linear_out"}) # baseline technologies
nts["ID"].add_tree(output["ID"]["basetech_inputs"], tree_name="ID_BX")

#### 1.2: End of pipe tree

In [5]:
nts["EOP"].add_tree(output["EOP"]["components"], tree_name = "EOP_CU", **{"type_f":"MNL"}) # C to U
nts["EOP"].add_tree(output["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # T to U 
nts["EOP"].add_tree(output["EOP"]["techs_inputs"], tree_name="EOP_TX") # T to inputs X

#### 1.3: Trees related to final goods and emissions account

*Read in tree and data from xlsx:*

In [6]:
nts['ID'].add_tree(directory['data']+'\\'+inputoth,tree_name='ID_Y',**{'sheet':'Y'})
DataBase.GPM_database.merge_dbs(nts['ID'].trees['ID_Y'].database,
                                excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Y': 'vars', 'M': 'vars','M_sets': 'sets'}), 'first')

*Add values written from xlsx to the set n:*

In [7]:
nts['ID'].trees['ID_Y'].database.update_all_sets(clean_up=False) # if variables are written to the database from the xlsx-file above, the domains are added to the relevant sets.

*Add new Q2P:*

In [8]:
output['ID']['Q2P'] = output['ID']['Q2P'].append(excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Q2P': 'maps'}).database['Q2P'].vals)

#### 1.3: Namespaces

*Give modules local namespaces:*

In [9]:
standard_sets = ('inp','out','int','wT','map_all','kno_out','kno_inp')
for module in modules:
    namespace = {k: module + '_' + k for k in standard_sets}
    nts[module].run_all(**namespace)
    #Also replaces keys with module-specific names, i.e. changes 'inp' to 'ID_inp' in the attributes/keys.
    for std_set in standard_sets:
        if hasattr(nts[module], std_set):
            setattr(nts[module], module + "_" + std_set, getattr(nts[module], std_set))
            delattr(nts[module], std_set)

## 2: Initialize model

*Initialize:*

In [10]:
gm = abatement.abate(nt=nts['ID'],tech=output,work_folder=directory['work'],**{'data_folder':directory['gams'],'name': 'atest'})
gm.initialize_variables(**{'check_variables': True})

*Add adhoc:*

In [11]:
gm.model.database['ID_mu_exo'] = gm.g('mu').index
gm.model.database['ID_mu_endoincalib'] = gm.g('mu').index.difference(gm.g('ID_mu_exo'))

*Check that all groups are written properly:*

In [12]:
from IPython.display import Markdown

In [20]:
var = gm.default_variables[1]
display(Markdown(f"**{var}+' exogenous:'**"))
display(gm.var_exo(var))
display(Markdown(f"**{var}+' endogenous:'**"))
display(gm.var_endo(var))

**PwT+' exogenous:'**

n
Y_K                        1
ID_t2_electricity          1
basetech_EL_oil            1
ID_t2_K                    1
basetech_EL_inp3           1
Y_inp3                     1
Y_oil                      1
basetech_EL_K              1
Y_electricity              1
basetech_EL_electricity    1
Name: PwT, dtype: int64

**PwT+' endogenous:'**

None

With these mappings:
* Define qSumU[n,nn] as the production by technology (n) of energy service (nn). This should be computed as: (NOTE: We define qsumU over n=e and nn = t instead):

$$
\begin{align}
    \text{qSumU[n,nn]} = \text{sum(nnn\$(e2t[nn,n] and e2u[nn,nnn]), qD[nnn])}.
\end{align}
$$

* Define the output share os[n,nn] as the share of output that technology $n$ produces of energy-service type $nn$. This should be defined as:

$$
\begin{align}
    \text{os[n,nn]} = \text{sum(nnn\$(e2u[nn,nnn] and u2t[nnn,n]), qD[nnn]) / qD[n]}.
\end{align}
$$

* Define variable qSumX[n,nn] defined as the use of aggregate input type $nn\in (electricity,oil,inp3,K)$ used to produce energy service $n$. This is defined as:

$$\begin{align}
    \text{qSumX[n,nn]} = \text{sum([nnn,nnnn]\$(e2ai2i[n,nn,nnn] and e2t[n,nnnn] and i2t[nnn,nnnn]), qD[nnn] * os[nnnn,n])}
\end{align}$$
* Here e2ai2i identifies the relevant quantities $qD[i]$ that are tied to relevant aggregate type of input, $e2t[e,t]$ identifies the correct output-share $os[t,e]$, and $i2t[i,t]$ ensures that only the relevant $i,t$ combinations are summed over.


Beyond these book keeping variables, we define a number of variables used in calibration:

* Define $currapp[n,nn]$ as the current share of coverage of a technology $n$ of energy-service $nn$. This is simply defined as (maybe we don't need to define the qSumU variable?)

$$\begin{align}
    curapp[n,nn] = qSumU[n,nn] / qD[nn].
\end{align}$$
* Define the level of application ... 